In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import numpy as np
import foolbox as fb
from collections import Counter
import matplotlib.pyplot as plt
from vgg_inference_model import VGG
import sys
sys.path.append("/home/reshikesh/hrushikesh/robust/vgg19/standard")
from vgg_change import VGG as AGG

batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = 'cpu'
print(device)

cifar_train = datasets.CIFAR10("~/data", train=True, download=True, transform=transforms.ToTensor())
cifar_test = datasets.CIFAR10("~/data", train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(cifar_train, batch_size = batch_size, shuffle=False)
test_loader = DataLoader(cifar_test, batch_size = batch_size, shuffle=True)


classes = ['aeroplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Load the inference model
inf_model = VGG(0.3).to(device)
inf_model.load_state_dict(torch.load("/home/reshikesh/hrushikesh/robust/vgg19/batchout_many/n_3/model_reg_121.pt"))
inf_model.eval()

# Create the foolbox model based on standard vgg. We use this to create adversarial images
preprocessing = dict(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616], axis=-3)
model = AGG()
model.eval()
model.load_state_dict(torch.load("/home/reshikesh/hrushikesh/robust/vgg19/batchout_many/n_3/model_reg_121.pt"))
fmodel = fb.PyTorchModel(model, bounds=(0,1), preprocessing=preprocessing)

# So there are two models now: Inference model which accepts two images and performs batchout and
# Foolbox model that will be used to get adversarial images

test_X, test_Y = next(iter(train_loader)) # Get 32 images/classes
test_X = test_X.to(device)
test_Y = test_Y.to(device)

cuda
Files already downloaded and verified
Files already downloaded and verified


In [11]:
# Perform the same with FGSM adversarial image
attack = fb.attacks.FGSM()
raw, clipped, is_adv = attack(fmodel, test_X, test_Y, epsilons=[8/255])
print(type(clipped), len(clipped))
print(type(clipped[0]), clipped[0].shape)
#len(clipped), print(type(clipped[0])))

<class 'list'> 1
<class 'torch.Tensor'> torch.Size([32, 3, 32, 32])


In [2]:
# Get the accuracy for all the images, so go through each batch
for batch_images, batch_labels in test_loader:
    batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)
    #raw, clipped, is_adv = attack(fmodel, test_X, test_Y, epsilons=[8/255])
    
    # Loop through each image in the batch
    mean_error = 0.0
    count_error = 0.0
    #for image, label in zip(clipped[0], batch_labels):
    for image, label in zip(batch_images, batch_labels):
        class_predictions = []
        predictions = torch.zeros(32, 10)
        # Loop through the images that we will perform batchout with the image
        for i, x in enumerate(test_X):

            # Pair the images to get a 4d tensor
            images = torch.stack((image, x))
            # Obtain the batchout prediction
            yp = inf_model(images)

            # Obtain the predicted class and append it to a list
            pred_class = classes[int(yp[0].argmax())]
            pred_class = yp[0].max(dim=0)[1]
            pred_class = int(pred_class.cpu().numpy())
            class_predictions.append(pred_class)

            # Store the predictions which will be later averaged
            predictions[i] = yp[0]

        # Get the mean predictions and class labels after batchout with all
        mean_predictions = predictions.mean(0)
        mean_pred_class = int(mean_predictions.max(dim=0)[1])

        count = Counter(class_predictions)
        common_pred_class = count.most_common()[0][0]

        count_error += common_pred_class != int(label)
        mean_error += mean_pred_class != int(label)

    print(count_error * 100 / 32, mean_error * 100 / 32)


18.75 34.375
15.625 31.25
21.875 37.5


KeyboardInterrupt: 

In [6]:
# Get the accuracy for all the images, so go through each batch
batch_images, batch_labels = next(iter(train_loader))
batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)
i = 0
x = test_X[i]
mean_error = 0.0
count_error = 0.0
for image, label in zip(batch_images, batch_labels):
    class_predictions = []
    print("Label of image is: ", label)
    predictions = torch.zeros(32, 10)
    # Loop through the images that we will perform batchout with the image

    # Pair the images to get a 4d tensor
    images = torch.stack((image, x))
    print(images.shape)
    # Obtain the batchout prediction
    yp = inf_model(images)
    print(yp)
    # Obtain the predicted class and append it to a list
    pred_class = classes[int(yp[0].argmax())]
    pred_class = yp[0].max(dim=0)[1]
    pred_class = int(pred_class.cpu().numpy())
    print("Predicted class: ",pred_class)
    class_predictions.append(pred_class)

    # Store the predictions which will be later averaged
    predictions[i] = yp[0]

    # Get the mean predictions and class labels after batchout with all
    mean_predictions = predictions.mean(0)
    print(mean_predictions)
    mean_pred_class = int(mean_predictions.max(dim=0)[1])

    count = Counter(class_predictions)
    common_pred_class = count.most_common()[0][0]

    count_error += common_pred_class != int(label)
    mean_error += mean_pred_class != int(label)
    print('-----------------')


Label of image is:  tensor(6, device='cuda:0')
torch.Size([2, 3, 32, 32])
tensor([[-0.4542, -0.2081, -0.2340,  0.0325,  0.2370, -0.4039,  1.8169,  0.1784,
         -0.5588, -0.2511],
        [-0.4542, -0.2081, -0.2340,  0.0325,  0.2370, -0.4039,  1.8169,  0.1784,
         -0.5588, -0.2511]], device='cuda:0', grad_fn=<AddmmBackward>)
Predicted class:  6
tensor([-0.0142, -0.0065, -0.0073,  0.0010,  0.0074, -0.0126,  0.0568,  0.0056,
        -0.0175, -0.0078], grad_fn=<MeanBackward1>)
-----------------
Label of image is:  tensor(9, device='cuda:0')
torch.Size([2, 3, 32, 32])
tensor([[ 2.1228,  1.9190, -1.2205, -1.7191, -2.2699, -1.7673, -1.8282, -1.6662,
          1.7356,  5.1342],
        [ 2.0537,  1.6667, -1.1055, -1.5303, -2.0597, -1.6417, -1.6582, -1.5417,
          1.6324,  4.6072]], device='cuda:0', grad_fn=<AddmmBackward>)
Predicted class:  9
tensor([ 0.0663,  0.0600, -0.0381, -0.0537, -0.0709, -0.0552, -0.0571, -0.0521,
         0.0542,  0.1604], grad_fn=<MeanBackward1>)
--------

Predicted class:  2
tensor([ 0.0052, -0.0472,  0.0723,  0.0053,  0.0327,  0.0140, -0.0222,  0.0170,
        -0.0301, -0.0357], grad_fn=<MeanBackward1>)
-----------------
Label of image is:  tensor(6, device='cuda:0')
torch.Size([2, 3, 32, 32])
tensor([[-0.4992, -0.5355, -0.1051,  0.3227,  0.2518, -0.1376,  2.3295,  0.0028,
         -0.7374, -0.6690],
        [-0.4986, -0.5343, -0.1082,  0.3243,  0.2573, -0.1405,  2.3278,  0.0037,
         -0.7365, -0.6719]], device='cuda:0', grad_fn=<AddmmBackward>)
Predicted class:  6
tensor([-1.5601e-02, -1.6734e-02, -3.2851e-03,  1.0084e-02,  7.8687e-03,
        -4.3006e-03,  7.2796e-02,  8.7496e-05, -2.3044e-02, -2.0907e-02],
       grad_fn=<MeanBackward1>)
-----------------
Label of image is:  tensor(3, device='cuda:0')
torch.Size([2, 3, 32, 32])
tensor([[-0.8363, -1.8670,  0.2250,  2.4614,  0.2975,  1.3602,  1.0379,  0.7077,
         -1.6973, -1.7546],
        [-0.8801, -1.5785,  0.3591,  1.1078,  0.2993,  0.6371,  2.5377,  0.4703,
         -1.54